In [10]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

### Data Description

U.S. Chronic Disease Indicators

Last Updated: March 9, 2024

Source: U.S. Chronic Disease Indicators - data.gov https://catalog.data.gov/dataset/u-s-chronic-disease-indicators

The dataset includes over 300,000 rows of chronic disease indicator data collected from U.S. states and territories. It encompasses data by topic (e.g., Cancer, Health Status), location, stratification (e.g., by race or gender), and confidence intervals for measurements.

### Load the Data

In [17]:
chronic_diseaseUS = pd.read_csv('U.S._Chronic_Disease_Indicators.csv')
chronic_diseaseUS.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DataValueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'Geolocation', 'LocationID', 'TopicID', 'QuestionID', 'ResponseID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

### Generate two tables of data as part of your database.

**Database Schema Plan**
To do that we will we split the chronic disease dataset  into two tables:

**Table 1: locations**
LocationID (Primary Key)

LocationAbbr

LocationDesc

Geolocation

**Table 2: chronic_data**
RecordID (Primary Key - auto-generated)

YearStart, YearEnd

LocationID (Foreign Key from locations)

Topic, Question, DataValue, LowConfidenceLimit, HighConfidenceLimit

StratificationCategory1, Stratification1

Next, I’ll set up the SQLite database, populate these tables, and perform the requested queries and operations.


In [3]:
# Create a new SQLite database
conn = sqlite3.connect('chronic_disease.db')
cursor = conn.cursor()

In [18]:
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS locations (
    LocationID INTEGER PRIMARY KEY,
    LocationAbbr TEXT NOT NULL,
    LocationDesc TEXT,
    Geolocation INTEGER
)''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS chronic_data ( 
RECORDID INTEGER PRIMARY KEY ,
    YearStart INTEGER, 
    YearEnd INTEGER ,
    LocationID TEXT NOT NULL,
    Topic TEXT , 
    Question TEXT,
    DataValue REAL, 
    LowConfidenceLimit REAL,
    HighConfidenceLimit REAL,
    StratificationCategory1 TEXT, 
    Stratification1 TEXT ,
    FOREIGN KEY (LocationID) REFERENCES authors(LocationID)
)
''')



### Add rows of data to the table

In [27]:
locations_df = chronic_diseaseUS[["LocationID", "LocationAbbr", "LocationDesc", "Geolocation"]]
locations_df

chronic_data = chronic_diseaseUS[[
    "YearStart", "YearEnd", "LocationID", "Geolocation",
    "Topic", "Question", "DataValue", "LowConfidenceLimit", "HighConfidenceLimit",
    "StratificationCategory1", "Stratification1"
]]
chronic_data = chronic_data.drop_duplicates()
locations_df = locations_df.drop_duplicates()


# Insert the  value into the location table
for index, row in locations_df.iterrows():
    cursor.execute('''
    INSERT OR IGNORE INTO locations (LocationID, LocationAbbr, LocationDesc, Geolocation) VALUES (?, ?, ?, ?)
    ''', (row['LocationID'], row['LocationAbbr'], row['LocationDesc'], row['Geolocation']))
    
    
# 2. Insert value to chronic_data into  table row by row
for index, row in chronic_data.iterrows():
    cursor.execute('''
    INSERT OR IGNORE INTO chronic_data (
        YearStart, YearEnd, LocationID,
        Topic, Question, DataValue, LowConfidenceLimit, HighConfidenceLimit,
        StratificationCategory1, Stratification1
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        row['YearStart'], row['YearEnd'], row['LocationID'], row['Topic'], row['Question'],
        row['DataValue'], row['LowConfidenceLimit'], row['HighConfidenceLimit'],
        row['StratificationCategory1'], row['Stratification1']
    ))
    

# 3. Commit the changes
conn.commit()

In [55]:
cursor.execute('''
SELECT *
    FROM locations;
''')

cursor.fetchall()

[(1, 'AL', 'Alabama', 'POINT (-86.63186076199969 32.84057112200048)'),
 (2, 'AK', 'Alaska', 'POINT (-147.72205903599973 64.84507995700051)'),
 (4, 'AZ', 'Arizona', 'POINT (-111.76381127699972 34.865970280000454)'),
 (5, 'AR', 'Arkansas', 'POINT (-92.27449074299966 34.74865012400045)'),
 (6, 'CA', 'California', 'POINT (-120.99999953799971 37.63864012300047)'),
 (8, 'CO', 'Colorado', 'POINT (-106.13361092099967 38.843840757000464)'),
 (9, 'CT', 'Connecticut', 'POINT (-72.64984095199964 41.56266102000046)'),
 (10, 'DE', 'Delaware', 'POINT (-75.57774116799965 39.008830667000495)'),
 (11, 'DC', 'District of Columbia', 'POINT (-77.036871 38.907192)'),
 (12, 'FL', 'Florida', 'POINT (-81.92896053899966 28.932040377000476)'),
 (13, 'GA', 'Georgia', 'POINT (-83.62758034599966 32.83968109300048)'),
 (15, 'HI', 'Hawaii', 'POINT (-157.85774940299973 21.304850435000446)'),
 (16, 'ID', 'Idaho', 'POINT (-114.3637300419997 43.682630005000476)'),
 (17, 'IL', 'Illinois', 'POINT (-88.99771017799969 40.485

In [31]:
cursor.execute('''
SELECT *
    FROM chronic_data;
''')

cursor.fetchall()

[(2019,
  2019,
  '5',
  'Diabetes',
  'Diabetes among adults',
  13.6,
  12.1,
  15.4,
  'Sex',
  'Male'),
 (2019,
  2019,
  '16',
  'Diabetes',
  'Diabetes among adults',
  10.6,
  9.1,
  12.2,
  'Sex',
  'Male'),
 (2019,
  2019,
  '18',
  'Sleep',
  'Short sleep duration among high school students',
  None,
  None,
  None,
  'Grade',
  'Grade 12'),
 (2019,
  2019,
  '19',
  'Asthma',
  'Asthma mortality among all people, underlying cause',
  54.0,
  None,
  None,
  'Overall',
  'Overall'),
 (2019,
  2019,
  '19',
  'Asthma',
  'Current asthma among adults',
  10.3,
  9.1,
  11.7,
  'Age',
  'Age 18-44'),
 (2019,
  2019,
  '19',
  'Diabetes',
  'Diabetes mortality among all people, underlying or contributing cause',
  54.0,
  None,
  None,
  'Age',
  'Age 0-44'),
 (2019,
  2019,
  '19',
  'Health Status',
  'Recent activity limitation among adults',
  2.3,
  2.1,
  2.5,
  'Sex',
  'Female'),
 (2019,
  2019,
  '19',
  'Mental Health',
  'Depression among adults',
  31.0,
  20.6,
  43.

### Connect the two tables with one key that is present in both databases.

In [42]:
# Create a new combined table
cursor.execute('''
CREATE TABLE IF NOT EXISTS chronic_with_location AS
SELECT 
    cd.YearStart,
    cd.YearEnd,
    cd.Topic,
    cd.Question,
    cd.DataValue,
    cd.LowConfidenceLimit,
    cd.HighConfidenceLimit,
    cd.StratificationCategory1,
    cd.Stratification1,
    l.LocationID,
    l.LocationAbbr,
    l.LocationDesc
FROM chronic_data cd
JOIN locations l ON cd.LocationID = l.LocationID
''')


In [43]:
# Run the JOIN query using cursor
cursor.execute('''
SELECT cd.YearStart, cd.Topic, cd.Question, cd.DataValue, l.LocationDesc
FROM chronic_data cd
JOIN locations l ON cd.LocationID = l.LocationID
LIMIT 5
''')

# Fetch the results
cursor.fetchall()


[(2019, 'Diabetes', 'Diabetes among adults', 13.6, 'Arkansas'),
 (2019, 'Diabetes', 'Diabetes among adults', 10.6, 'Idaho'),
 (2019,
  'Sleep',
  'Short sleep duration among high school students',
  None,
  'Indiana'),
 (2019,
  'Asthma',
  'Asthma mortality among all people, underlying cause',
  54.0,
  'Iowa'),
 (2019, 'Asthma', 'Current asthma among adults', 10.3, 'Iowa')]

In [44]:
# Perform JOIN between chronic_data and locations
cursor.execute('''
SELECT 
    cd.YearStart,
    cd.Topic,
    cd.Question,
    cd.DataValue,
    l.LocationDesc
FROM chronic_data cd
JOIN locations l ON cd.LocationID = l.LocationID
LIMIT 5
''')
# Fetch and print the result
cursor.fetchall()




[(2019, 'Diabetes', 'Diabetes among adults', 13.6, 'Arkansas'),
 (2019, 'Diabetes', 'Diabetes among adults', 10.6, 'Idaho'),
 (2019,
  'Sleep',
  'Short sleep duration among high school students',
  None,
  'Indiana'),
 (2019,
  'Asthma',
  'Asthma mortality among all people, underlying cause',
  54.0,
  'Iowa'),
 (2019, 'Asthma', 'Current asthma among adults', 10.3, 'Iowa')]

### Remove records with null values from your tables.


In [47]:
cursor.execute('''
DELETE FROM chronic_data
WHERE YearStart IS NULL OR YearEnd IS NULL OR LocationID IS NULL  OR Topic IS NULL OR Question IS NULL OR
      DataValue IS NULL OR LowConfidenceLimit IS NULL OR HighConfidenceLimit IS NULL OR
      StratificationCategory1 IS NULL OR Stratification1 IS NULL
''')


cursor.execute('''
DELETE FROM locations
WHERE LocationID IS NULL OR LocationAbbr IS NULL OR LocationDesc IS NULL OR Geolocation IS NULL
''')
conn.commit()


### Perform two different queries of your data using SQL methods.
Run Two SQL Queries with Aggregations/Organization

In [53]:

# Run the query using cursor
cursor.execute('''
SELECT Topic, ROUND(AVG(DataValue), 2) AS AvgValue
FROM chronic_data
GROUP BY Topic
ORDER BY AvgValue DESC
LIMIT 10
''')

# Fetch the results
cursor.fetchall()

[('Cancer', 100.52),
 ('Maternal Health', 87.88),
 ('Chronic Obstructive Pulmonary Disease', 78.14),
 ('Cardiovascular Disease', 76.66),
 ('Oral Health', 52.55),
 ('Sleep', 46.79),
 ('Diabetes', 46.43),
 ('Immunization', 44.91),
 ('Social Determinants of Health', 44.61),
 ('Arthritis', 31.9)]

In [54]:

# Run the query using cursor
cursor.execute('''
SELECT Topic, ROUND(AVG(DataValue), 2) AS AvgValue
FROM chronic_data
GROUP BY Topic
ORDER BY AvgValue DESC
LIMIT 10
''')

# Fetch the results
cursor.fetchall()

[('Cancer', 100.52),
 ('Maternal Health', 87.88),
 ('Chronic Obstructive Pulmonary Disease', 78.14),
 ('Cardiovascular Disease', 76.66),
 ('Oral Health', 52.55),
 ('Sleep', 46.79),
 ('Diabetes', 46.43),
 ('Immunization', 44.91),
 ('Social Determinants of Health', 44.61),
 ('Arthritis', 31.9)]